# Section 2: Building a Wallet Portfolio Manager

In this section, you'll learn to:
- Calculate current wallet net worth
- Build portfolio performance charts
- Analyze individual token holdings
- Create interactive OHLCV charts
- Implement real-time WebSocket updates
- Display live transaction feeds

## APIs & WebSockets Used in This Section

### REST API Endpoints:
- **`GET /wallet/v2/current-net-worth`** - Get current wallet portfolio value and token breakdown
- **`GET /wallet/v2/net-worth`** - Retrieve historical net worth data for performance tracking
- **`GET /wallet/v2/net-worth-details`** - Access detailed token holdings and balances
- **`GET /defi/v3/ohlcv`** - Fetch OHLCV (candlestick) data for advanced charting
- **`GET /defi/price`** - Get current token prices for real-time updates

### WebSocket Subscriptions:
- **`SUBSCRIBE_PRICE`** - Real-time price updates for multiple tokens
- **`SUBSCRIBE_TXS`** - Live transaction feeds for specific tokens
- **Price Monitoring** - Continuous price tracking with timestamps
- **Transaction Streaming** - Real-time buy/sell activity notifications

### Response Fields You'll Work With:
- **Current Net Worth**: `total_value`, `items[]` with `symbol`, `value`, `balance`, `price`
- **Net Worth History**: `history[]` with `timestamp`, `total_value`
- **OHLCV Data**: `items[]` with `o`, `h`, `l`, `c`, `v`, `unixTime`
- **WebSocket Price**: `address`, `value`, `updateUnixTime`
- **WebSocket Transactions**: `txType`, `amount`, `price`, `txHash`, `timestamp`

Let's build a professional-grade portfolio management system!

## 🛠 Setup and Imports

**Important**: This section requires a **Business API Key** from Birdeye Data Services.

### Getting Your Business API Key

The wallet portfolio features require access to premium endpoints that are only available with a Business package. Your workshop host will provide you with a Business API key.

**Add the Business API key to your .env file:**
```
BDS_API_KEY=your_business_api_key_here
```

If you don't have a Business API key, you can:
1. Contact your workshop host for a temporary key
2. Sign up for a Business plan at [Birdeye Data Services](https://bds.birdeye.so)
3. Continue with the demo to see how the features work

In [1]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta
import json
import time
import threading
import warnings
warnings.filterwarnings('ignore')

# Force reload of utils module to ensure latest version
import utils
import importlib
importlib.reload(utils)

# Import our custom utilities
from utils import (
    BirdeyeDataServices, 
    BirdeyeDataServicesWebSocket,
    create_portfolio_chart,
    create_portfolio_pie_chart,
    create_candlestick_chart,
    format_currency,
    format_transaction_data,
    check_api_key
)

print("✅ All modules imported successfully, including create_portfolio_pie_chart!")

# Check Business API key and initialize client
if check_api_key('business'):
    # Initialize Birdeye Data Services client with Business API key
    birdeye = BirdeyeDataServices(api_key_type='business')
    print("Setup complete! Ready to analyze portfolios with Business API.")
else:
    print("Please configure BDS_API_KEY in your .env file")
    print("Contact your workshop host for a Business API key")

✅ All modules imported successfully, including create_portfolio_pie_chart!
✅ BDS_API_KEY found!
Setup complete! Ready to analyze portfolios with Business API.


### Troubleshooting Note

**If you encounter import errors for `create_portfolio_pie_chart`:**
1. **Restart the kernel**: Go to `Kernel` → `Restart` in the menu
2. **Re-run the import cell above**
3. **Clear browser cache** if the issue persists

The function exists in `utils.py` and the reload code above should resolve any caching issues.

## Step 1: Set Up Wallet Address

Let's configure a wallet address for portfolio analysis:

In [ ]:
# Example wallet addresses (you can replace with any Solana wallet address)
EXAMPLE_WALLETS = {
    "Example Wallet 1": "4Be9CvxqHW6BYiRAxW9Q3xu1ycTMWaL5z8NX4HR3ha7t",
    "Example Wallet 2": "h9QLBKdx5VbPFoEUc8pfejqTH5PhCVo3ewwrrWiikGb",
    "Example Wallet 3": "6qBogbNZYGvzSb1UfGU7BPggDgFbb34DEr3fmCkFaf8T"
}

# Create wallet selector
wallet_dropdown = widgets.Dropdown(
    options=[(name, addr) for name, addr in EXAMPLE_WALLETS.items()],
    value=list(EXAMPLE_WALLETS.values())[0],
    description='Select Wallet:',
    style={'description_width': 'initial'}
)

custom_wallet_input = widgets.Text(
    value='',
    placeholder='Or enter custom wallet address',
    description='Custom Wallet:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

def get_selected_wallet():
    if custom_wallet_input.value.strip():
        return custom_wallet_input.value.strip()
    return wallet_dropdown.value

print("🎯 Wallet Configuration:")
display(widgets.VBox([wallet_dropdown, custom_wallet_input]))

# Set initial wallet
current_wallet = get_selected_wallet()
print(f"\n📍 Current wallet: {current_wallet}")

## Step 2: Get Current Net Worth

Let's fetch the current net worth of the selected wallet:

In [ ]:
# Get current wallet net worth
current_wallet = get_selected_wallet()

print(f"💰 Fetching current net worth for wallet: {current_wallet}")

net_worth_data = birdeye.get_wallet_net_worth(current_wallet)

if net_worth_data and 'data' in net_worth_data:
    # Use correct field names from API response
    data = net_worth_data['data']
    total_usd = float(data.get('total_value', 0))
    
    print(f"\nCurrent Portfolio Value: {format_currency(total_usd)}")
    print("-" * 50)
    
    # Display breakdown if available
    if 'items' in data and data['items']:
        items = data['items']
        print(f"Portfolio Breakdown ({len(items)} tokens):")
        
        for item in items[:10]:  # Show top 10 holdings
            symbol = item.get('symbol', 'Unknown')
            value_usd = float(item.get('value', 0))
            balance = int(float(item.get('balance', 0)))
            percentage = (value_usd / total_usd * 100) if total_usd > 0 else 0
            
            print(f"   • {symbol}: {format_currency(value_usd)} ({percentage:.1f}%) - {balance:.4f} tokens")
    else:
        print("No individual token holdings found")
    
    print("\nNet worth data loaded successfully!")
else:
    print("Could not fetch net worth data. The wallet might be empty or invalid.")
    total_usd = 0

## Step 3: Portfolio Performance Chart

Let's visualize the portfolio's performance over time:

In [ ]:
# Get net worth history
print(f"📈 Fetching portfolio history for wallet: {current_wallet}")
net_worth_history = birdeye.get_wallet_net_worth_history(current_wallet)

if net_worth_history and 'data' in net_worth_history:
    # Create portfolio performance chart
    portfolio_chart = create_portfolio_chart(net_worth_history)
    
    if portfolio_chart:
        print("\n📊 Portfolio Performance Over Time:")
        portfolio_chart.show()
        
        # Calculate performance metrics
        history_data = net_worth_history['data']
        if len(history_data) > 1:
            first_value = history_data[0].get('total_value', 0)
            last_value = history_data[-1].get('total_value', 0)
            
            if first_value > 0:
                total_return = ((last_value - first_value) / first_value) * 100
                print(f"\n📊 Performance Metrics:")
                print(f"   📈 Total Return: {total_return:.2f}%")
                print(f"   💰 Value Change: {format_currency(last_value - first_value)}")
                
                # Find highest and lowest points
                values = [item.get('total_value', 0) for item in history_data]
                max_value = max(values)
                min_value = min(values)
                
                print(f"   🔺 All-time High: {format_currency(max_value)}")
                print(f"   🔻 All-time Low: {format_currency(min_value)}")
    else:
        print("❌ Could not create portfolio chart")
else:
    print("❌ Could not fetch portfolio history")

## Step 4: OHLCV Candlestick Charts

Let's create professional candlestick charts for detailed price analysis:

In [ ]:
# Create OHLCV chart for SOL (as an example)
sol_address = "So11111111111111111111111111111111111111112"

print("📊 Creating OHLCV candlestick chart for SOL...")

# Get OHLCV data
ohlcv_data = birdeye.get_ohlcv_data(sol_address, type_="1D")

if ohlcv_data:
    # Create candlestick chart
    candlestick_chart = create_candlestick_chart(ohlcv_data, "SOL")
    
    if candlestick_chart:
        print("\n📈 SOL OHLCV Candlestick Chart:")
        candlestick_chart.show()
    else:
        print("❌ Could not create candlestick chart")
else:
    print("❌ Could not fetch OHLCV data")

print("\n💡 Candlestick Chart Features:")
print("   🟢 Green candles: Price increased (Close > Open)")
print("   🔴 Red candles: Price decreased (Close < Open)")
print("   📊 Volume bars: Trading activity")
print("   🔍 Interactive: Zoom, pan, and hover for details")

## Step 5: Real-time WebSocket Updates

Now let's implement real-time price updates using WebSocket connections:

In [ ]:
# Real-time price monitoring with WebSocket
class RealTimePriceMonitor:
    def __init__(self):
        self.ws = None
        self.price_data = {}
        self.is_running = False
        self.output_widget = widgets.Output()
        
    def start_monitoring(self, token_addresses):
        """Start real-time price monitoring"""
        if self.is_running:
            print("⚠️ Monitoring already running")
            return
            
        self.is_running = True
        
        # Initialize WebSocket
        self.ws = BirdeyeDataServicesWebSocket()
        
        # Set up price update callback
        def on_price_update(data):
            if 'data' in data:
                address = data['data'].get('address')
                price = data['data'].get('value')
                
                if address and price:
                    self.price_data[address] = {
                        'price': price,
                        'timestamp': datetime.now()
                    }
                    
                    with self.output_widget:
                        clear_output(wait=True)
                        print(f"🔄 Real-time Price Updates:")
                        print(f"{'='*50}")
                        
                        for addr, info in self.price_data.items():
                            symbol = 'SOL' if addr == 'So11111111111111111111111111111111111111112' else addr[:8]
                            timestamp = info['timestamp'].strftime('%H:%M:%S')
                            print(f"💰 {symbol}: ${info['price']:.6f} (Updated: {timestamp})")
        
        self.ws.add_callback('PRICE_DATA', on_price_update)
        
        # Connect and subscribe
        try:
            self.ws.connect()
            
            for address in token_addresses:
                self.ws.subscribe_price(address)
                print(f"📡 Subscribed to price updates for {address[:8]}...")
                
        except Exception as e:
            print(f"❌ WebSocket connection failed: {e}")
            self.is_running = False
    
    def stop_monitoring(self):
        """Stop real-time monitoring"""
        if self.ws:
            self.ws.disconnect()
        self.is_running = False
        print("🛑 Real-time monitoring stopped")
    
    def get_output_widget(self):
        return self.output_widget

# Create price monitor instance
price_monitor = RealTimePriceMonitor()

# Control buttons
start_button = widgets.Button(
    description='Start Real-time Monitoring',
    button_style='success',
    tooltip='Start WebSocket price monitoring'
)

stop_button = widgets.Button(
    description='Stop Monitoring',
    button_style='danger',
    tooltip='Stop WebSocket monitoring'
)

def on_start_click(b):
    # Monitor SOL and USDC prices
    tokens_to_monitor = [
        "So11111111111111111111111111111111111111112",  # SOL
        "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"   # USDC
    ]
    price_monitor.start_monitoring(tokens_to_monitor)

def on_stop_click(b):
    price_monitor.stop_monitoring()

start_button.on_click(on_start_click)
stop_button.on_click(on_stop_click)

print("🔄 Real-time Price Monitoring:")
display(widgets.HBox([start_button, stop_button]))
display(price_monitor.get_output_widget())

print("\n💡 WebSocket Features:")
print("   📡 Real-time price updates")
print("   🔄 Live transaction feeds")
print("   ⚡ Low latency data")
print("   🎯 Subscribe to specific tokens")

## Step 6: Live Transaction Feed

Let's create a live transaction feed for monitoring wallet activity:

In [ ]:
# Live transaction monitoring
class TransactionMonitor:
    def __init__(self):
        self.ws = None
        self.transactions = []
        self.is_running = False
        self.output_widget = widgets.Output()
        
    def start_monitoring(self, token_address):
        """Start monitoring transactions for a token"""
        if self.is_running:
            print("⚠️ Transaction monitoring already running")
            return
            
        self.is_running = True
        self.transactions = []
        
        # Initialize WebSocket
        self.ws = BirdeyeDataServicesWebSocket()
        
        # Set up transaction callback
        def on_transaction(data):
            if 'data' in data:
                tx_data = data['data']
                self.transactions.append({
                    'timestamp': datetime.now(),
                    'type': tx_data.get('txType', 'Unknown'),
                    'amount': tx_data.get('amount', 0),
                    'price': tx_data.get('price', 0),
                    'signature': tx_data.get('txHash', '')[:10] + '...'
                })
                
                # Keep only last 10 transactions
                if len(self.transactions) > 10:
                    self.transactions = self.transactions[-10:]
                
                with self.output_widget:
                    clear_output(wait=True)
                    print(f"📱 Live Transaction Feed:")
                    print(f"{'='*60}")
                    
                    for tx in reversed(self.transactions):  # Show newest first
                        timestamp = tx['timestamp'].strftime('%H:%M:%S')
                        tx_type = tx['type']
                        amount = tx['amount']
                        price = tx['price']
                        signature = tx['signature']
                        
                        print(f"🔄 [{timestamp}] {tx_type}: {amount:.4f} @ ${price:.6f} ({signature})")
        
        self.ws.add_callback('TRANSACTION_DATA', on_transaction)
        
        # Connect and subscribe
        try:
            self.ws.connect()
            self.ws.subscribe_transactions(token_address)
            print(f"📡 Subscribed to transactions for {token_address[:8]}...")
            
        except Exception as e:
            print(f"❌ WebSocket connection failed: {e}")
            self.is_running = False
    
    def stop_monitoring(self):
        """Stop transaction monitoring"""
        if self.ws:
            self.ws.disconnect()
        self.is_running = False
        print("🛑 Transaction monitoring stopped")
    
    def get_output_widget(self):
        return self.output_widget

# Create transaction monitor
tx_monitor = TransactionMonitor()

# Control buttons for transaction monitoring
start_tx_button = widgets.Button(
    description='Start Transaction Feed',
    button_style='info',
    tooltip='Start monitoring live transactions'
)

stop_tx_button = widgets.Button(
    description='Stop Transaction Feed',
    button_style='warning',
    tooltip='Stop transaction monitoring'
)

def on_start_tx_click(b):
    # Monitor SOL transactions
    sol_address = "So11111111111111111111111111111111111111112"
    tx_monitor.start_monitoring(sol_address)

def on_stop_tx_click(b):
    tx_monitor.stop_monitoring()

start_tx_button.on_click(on_start_tx_click)
stop_tx_button.on_click(on_stop_tx_click)

print("📱 Live Transaction Monitoring:")
display(widgets.HBox([start_tx_button, stop_tx_button]))
display(tx_monitor.get_output_widget())

print("\n💡 Transaction Feed Features:")
print("   📱 Real-time transaction updates")
print("   🔄 Buy/Sell activity tracking")
print("   💰 Price and volume information")
print("   🔍 Transaction signature links")

## Step 7: Interactive Portfolio Analyzer

Let's create an interactive tool to analyze any wallet's portfolio with custom time selection and pie chart visualization:

In [ ]:
# Interactive Portfolio Analyzer with Pie Chart
# Force reload to ensure we have the latest version
import importlib
import utils
importlib.reload(utils)
from utils import create_portfolio_pie_chart
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

class PortfolioAnalyzer:
    def __init__(self):
        self.output_widget = widgets.Output()
        
        # Wallet address input
        self.wallet_input = widgets.Text(
            value='9WzDXwBbmkg8ZTbNMqUxvQRAyrZzDsGYdLVL9zYtAWWM',
            placeholder='Enter wallet address',
            description='Wallet Address:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        
        # Time selection dropdown
        self.time_select = widgets.Dropdown(
            options=[
                ('Current Time', 'current'),
                ('1 Hour Ago', '1h'),
                ('1 Day Ago', '1d'),
                ('1 Week Ago', '1w'),
                ('1 Month Ago', '1m')
            ],
            value='current',
            description='Time:',
            style={'description_width': 'initial'}
        )
        
        # Analyze button
        self.analyze_button = widgets.Button(
            description='Analyze Portfolio',
            button_style='primary',
            tooltip='Analyze wallet portfolio'
        )
        
        # History chart button
        self.history_button = widgets.Button(
            description='Show History Chart',
            button_style='info',
            tooltip='Show net worth history'
        )
        
        self.analyze_button.on_click(self.analyze_portfolio)
        self.history_button.on_click(self.show_history)
    
    def analyze_portfolio(self, b):
        with self.output_widget:
            clear_output(wait=True)
            
            wallet_address = self.wallet_input.value.strip()
            time_selection = self.time_select.value
            
            if not wallet_address:
                print("Please enter a wallet address")
                return
            
            print(f"Analyzing portfolio for: {wallet_address}")
            print(f"Time selection: {self.time_select.label}")
            print("-" * 60)
            
            # Get current net worth details
            if time_selection == 'current':
                net_worth_data = birdeye.get_wallet_net_worth_details(wallet_address)
            else:
                # For historical data, we'll use current data as demo
                # In a real implementation, you'd use historical endpoints
                print(f"Note: Using current data for {self.time_select.label} (historical data requires additional API calls)")
                net_worth_data = birdeye.get_wallet_net_worth_details(wallet_address)
            
            if net_worth_data and 'data' in net_worth_data:
                data = net_worth_data['data']
                total_value = data.get('total_value', 0)
                
                print(f"Total Portfolio Value: {format_currency(total_value)}")
                
                if 'items' in data and data['items']:
                    items = data['items']
                    print(f"Number of tokens: {len(items)}")
                    print("\nTop 5 Holdings:")
                    
                    for i, item in enumerate(items[:5], 1):
                        symbol = item.get('symbol', 'Unknown')
                        value = item.get('value', 0)
                        balance = item.get('balance', 0)
                        percentage = (value / total_value * 100) if total_value > 0 else 0
                        
                        print(f"   {i}. {symbol}: {format_currency(value)} ({percentage:.1f}%) - {balance:.4f} tokens")
                    
                    # Create pie chart
                    print("\nCreating portfolio allocation pie chart...")
                    pie_chart = create_portfolio_pie_chart(net_worth_data, f"Portfolio Allocation - {wallet_address}")
                    
                    if pie_chart:
                        pie_chart.show()
                    else:
                        print("Could not create pie chart")
                else:
                    print("No token holdings found")
            else:
                print("Could not fetch portfolio data")
    
    def show_history(self, b):
        with self.output_widget:
            clear_output(wait=True)
            
            wallet_address = self.wallet_input.value.strip()
            
            if not wallet_address:
                print("Please enter a wallet address")
                return
            
            print(f"Fetching net worth history for: {wallet_address}")
            print("-" * 60)
            
            # Get net worth history
            history_data = birdeye.get_wallet_net_worth_history(wallet_address)
            
            if history_data:
                # Create line chart
                history_chart = create_portfolio_chart(history_data)
                
                if history_chart:
                    history_chart.update_layout(title=f"Net Worth History - {wallet_address}")
                    history_chart.show()
                    print("Net worth history chart displayed above")
                else:
                    print("Could not create history chart (likely no historical data available)")
            else:
                print("Could not fetch net worth history")
    
    def display(self):
        return widgets.VBox([
            self.wallet_input,
            self.time_select,
            widgets.HBox([self.analyze_button, self.history_button]),
            self.output_widget
        ])

# Create and display the portfolio analyzer
print("Interactive Portfolio Analyzer:")
analyzer = PortfolioAnalyzer()
display(analyzer.display())

print("\nFeatures:")
print("• Enter any Solana wallet address")
print("• Select time period (current time is most accurate)")
print("• View portfolio allocation as pie chart")
print("• Display net worth history as line chart")
print("• Analyze top token holdings")

## Step 8: Interactive Portfolio Dashboard

Let's create a comprehensive dashboard that combines all our features:

In [ ]:
# Comprehensive Portfolio Dashboard
def create_portfolio_dashboard():
    """Create an interactive portfolio dashboard"""
    
    # Dashboard output widget
    dashboard_output = widgets.Output()
    
    # Refresh button
    refresh_button = widgets.Button(
        description='🔄 Refresh Dashboard',
        button_style='primary',
        tooltip='Refresh all portfolio data'
    )
    
    def refresh_dashboard(b=None):
        with dashboard_output:
            clear_output(wait=True)
            
            current_wallet = get_selected_wallet()
            
            print(f"🎮 Portfolio Dashboard")
            print(f"{'='*60}")
            print(f"📍 Wallet: {current_wallet[:10]}")
            print(f"🕐 Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print()
            
            # Get current net worth
            print("💰 Fetching current portfolio value...")
            net_worth = birdeye.get_wallet_net_worth(current_wallet)
            
            if net_worth and 'data' in net_worth:
                total_value = net_worth['data'].get('total_value', 0)
                print(f"💎 Total Portfolio Value: {format_currency(total_value)}")
                
                if 'items' in net_worth['data']:
                    items = net_worth['data']['items']
                    print(f"\n📊 Top Holdings ({len(items)} total):")
                    
                    for i, item in enumerate(items[:5], 1):
                        symbol = item.get('symbol', 'Unknown')
                        value = item.get('value', 0)  # Use 'value' instead of 'valueUsd'
                        balance = item.get('balance', 0)
                        percentage = (value / total_value * 100) if total_value > 0 else 0
                        
                        print(f"   {i}. {symbol}: {format_currency(value)} ({percentage:.1f}%)")
                        print(f"      Balance: {balance:.4f} tokens")
            else:
                print("❌ Could not fetch portfolio data")
            
            # Get recent activity
            print("\n🔄 Recent Activity:")
            print("   📡 WebSocket connections available for real-time updates")
            print("   📱 Transaction monitoring ready")
            print("   📊 Charts and analytics loaded")
            
            print("\n✅ Dashboard updated successfully!")
    
    refresh_button.on_click(refresh_dashboard)
    
    # Initial load
    refresh_dashboard()
    
    return widgets.VBox([refresh_button, dashboard_output])

# Create and display dashboard
print("🎮 Creating Interactive Portfolio Dashboard...")
dashboard = create_portfolio_dashboard()
display(dashboard)

## 🎉 Congratulations!

You've successfully built a comprehensive wallet portfolio management system! Here's what you've accomplished:

### What You've Built:
- **Portfolio Valuation**: Real-time net worth calculation using `/wallet/v2/current-net-worth`
- **Performance Tracking**: Historical portfolio charts with `/wallet/v2/net-worth`
- **Advanced Charts**: OHLCV candlestick charts using `/defi/v3/ohlcv`
- **Real-time Updates**: WebSocket integration for live price feeds
- **Transaction Monitoring**: Live transaction feeds with WebSocket subscriptions
- **Interactive Dashboard**: Comprehensive portfolio management interface

### Advanced Skills Developed:
- WebSocket programming for real-time data
- Advanced data visualization techniques
- Interactive widget development
- Portfolio analytics and metrics
- Professional dashboard design

### Production-Ready Features:
- Error handling and graceful degradation
- Real-time data streaming
- Interactive user interfaces
- Professional data presentation
- Scalable architecture

### Next Level Ideas:
- Add portfolio alerts and notifications
- Implement portfolio rebalancing suggestions
- Create custom trading strategies
- Build multi-wallet management
- Add DeFi protocol integration
- Implement risk management tools

### Workshop Complete!

You now have the skills to build professional-grade blockchain data applications using Birdeye Data Services. The combination of REST APIs and WebSocket connections gives you everything needed for real-time financial applications.

**Key Takeaways**:
- **API Integration**: Master both REST and WebSocket protocols
- **Data Visualization**: Create compelling charts and dashboards
- **User Experience**: Build interactive and responsive interfaces
- **Production Ready**: Handle errors, edge cases, and real-world scenarios

Great work! You're now ready to build the next generation of blockchain data applications! 🎊

### Additional Resources:
- [Birdeye API Documentation](https://docs.birdeye.so)
- [WebSocket Guide](https://docs.birdeye.so/docs/websocket)